In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [5]:
training_data = datasets.FashionMNIST(root = "../../dat/pyt", train=True, download=True, transform=ToTensor())
test_data = datasets.FashionMNIST(root = "../../dat/pyt", train=False, download=True, transform=ToTensor())

In [6]:
batch_size = 64
# create data loaders
training_data_loader = DataLoader(training_data, batch_size=batch_size)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_data_loader:
  print("Shape of X [N, C, H, W]: ", X.shape)
  print("Shape of y: ", y.shape, y.dtype)
  break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [7]:
torch.cuda.is_available()

False

In [8]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device.")

Using cpu device.


In [9]:
# Define model
class NeuralNetwork(nn.Module):
  def __init__(self) -> None:
      super().__init__()
      self.flatten = nn.Flatten()
      self.model = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 256),
        nn.ReLU(),
        nn.Linear(256, 10)
      )
  def forward(self, x):
    x = self.flatten(x)
    logits = self.model(x)
    return logits

In [10]:
model = NeuralNetwork().to(device)

In [11]:
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (model): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=256, bias=True)
    (3): ReLU()
    (4): Linear(in_features=256, out_features=10, bias=True)
  )
)


In [12]:
optimizer = torch.optim.Adam(model.parameters())

In [16]:
criterion = nn.CrossEntropyLoss()

In [17]:
def train(dataloader, model, criterion, optimizer):
  N = len(dataloader.dataset)
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)
    prediction = model(X)
    loss = criterion(prediction, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if batch % 100 == 0:
      loss, current = loss.item(), batch * len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{N:>5d}]")

In [41]:
def test(dataloader, model, criterion):
  N = len(dataloader.dataset)
  num_batch = len(dataloader)
  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      prediction = model(X)
      test_loss += criterion(prediction, y).item()
      correct += (prediction.argmax(1) == y).type(torch.float).sum().item()
  test_loss /= num_batch
  correct /= N
  print(f"Test accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [20]:
epochs = 15
for t in range(epochs):
  print(f"Epoch {t+1}\n-------")
  train(training_data_loader, model, criterion, optimizer)
  test(test_data_loader, model, criterion)
print("Done.")

Epoch 1
-------
loss: 0.206834 [    0/60000]
loss: 0.258138 [ 6400/60000]
loss: 0.193781 [12800/60000]
loss: 0.229687 [19200/60000]
loss: 0.325572 [25600/60000]
loss: 0.281255 [32000/60000]
loss: 0.210263 [38400/60000]
loss: 0.348169 [44800/60000]
loss: 0.285773 [51200/60000]
loss: 0.342290 [57600/60000]
Test error: 
Accuracy: 87.4%, Avg loss: 0.357545 

Epoch 2
-------
loss: 0.178744 [    0/60000]
loss: 0.193994 [ 6400/60000]
loss: 0.180524 [12800/60000]
loss: 0.226158 [19200/60000]
loss: 0.289966 [25600/60000]
loss: 0.271724 [32000/60000]
loss: 0.197576 [38400/60000]
loss: 0.305221 [44800/60000]
loss: 0.235226 [51200/60000]
loss: 0.341489 [57600/60000]
Test error: 
Accuracy: 88.0%, Avg loss: 0.341133 

Epoch 3
-------
loss: 0.152790 [    0/60000]
loss: 0.169653 [ 6400/60000]
loss: 0.215673 [12800/60000]
loss: 0.226801 [19200/60000]
loss: 0.317692 [25600/60000]
loss: 0.233184 [32000/60000]
loss: 0.199444 [38400/60000]
loss: 0.249745 [44800/60000]
loss: 0.231852 [51200/60000]
loss: 0.3

In [22]:
# save the model
torch.save(model.state_dict(), "../../dat/pyt/FashionMNIST/model")

In [23]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

In [40]:
model.eval()
x, y = test_data[100][0], test_data[100][1]
with torch.no_grad():
  z = model(x)
  predicted = z[0].argmax(0)
  print(f"predicted: '{classes[predicted]}', actual: '{classes[y]}'")

predicted: 'Dress', actual: 'Dress'
